# Vision

The purpose of this notebook is to create images that will make the workflow of moving apples easy to do. 
The final product will be a Streamlit site where I can go forward and backwards and see frames and the transitions between them.

Requirements: Original frame, downsampled frame  with coordinates of every apple, previous downsampled frame, and a diff frame with the added apples in green and removed apples in red.

In [1]:
import cv2
import numpy as np
from pathlib import Path
import os

In [4]:
badapple_path = Path(".").resolve().parent / "badapple-small.mp4"

cap = cv2.VideoCapture(str(badapple_path))

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"width: {width}, height: {height}, fps: {fps}, n_frames: {n_frames}")

square_side = 30

width: 960, height: 720, fps: 30, n_frames: 6572


In [3]:
# Generate real_frames
ret, frame = cap.read()
index = 0
while ret:
    cv2.imwrite(f"real_frames/{index}.jpg", frame)
    ret, frame = cap.read()
    index += 1

In [10]:
# generate downsampled frames with coordinates of apples labeled
# reset cap
cap = cv2.VideoCapture(str(badapple_path))
index = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # convert to grayscale
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if index % 100 == 0 or index == n_frames - 1:
        print(f"frame {index}/{n_frames}", end="\r")
    out_frame = np.ones((height, width), dtype=np.uint8) * 255

    current_frame_pixels = set()
    for i in range(0, width - 1, square_side):
        for j in range(0, height - 1, square_side):
            section = frame[j: j + square_side, i: i + square_side]
            # if its dark, add dark circle with white coordinate text
            if (np.mean(section)) < 50:
                cv2.circle(out_frame, (i+square_side//2, j+square_side//2), square_side//2, 0, -1)
                text_color = 255
            else:
                text_color = 0
            # otherwise, black coordinate text
            # put first coordinate higher, second coordinate lower
            
            cv2.putText(out_frame, f"{j//square_side+1}", (i+square_side//4, j + square_side -10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
            cv2.putText(out_frame, f"{i//square_side+1}", (i+square_side//4, j + square_side - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
    cv2.imwrite(f"circle_frames/{index}.jpg", out_frame)
    index += 1

                
    
